In the first two assignments, we have learned how to infer part based components (known as mutational signatures) generated by particular mutational processes using Non-negative Matrix Factorization (NMF). By doing this, we are trying to reconstruct the mutation catalog in a given sample with mutational signatures and their contributions.

In this group project, you will use similar mutational profiles and signature activities to predict cancer types but with much larger sample size. 
You should:
* Separate the data into training and test groups within each cancer type.
* Find out which features are informative for the prediction of the cancer type (label). You should try both combining the profiles with activities and using each data type independently.
* Implement different models of your choice for classification of the samples given the input data and evaluate the model performance using test data to avoid overfitting. Explain briefly how does each model that you have used work.
* Report model performance, using standard machine learning metrics such as confusion matrices etc. 
* Compare model performance across methods and across cancer types, are some types easier to predict than others.
* Submit a single Jupyter notebook as the final report and present that during the last assignment session.

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingClassifier
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report

def format_channel_names(df):
    """Converts Trinucleotide and Mutation type to A[C>A]A format."""
    def transform(row):
        tri = row['Trinucleotide']
        mut = row['Mutation type']
        return f"{tri[0]}[{mut}]{tri[2]}"
    
    return df.apply(transform, axis=1)

def load_and_preprocess(catalog_files, activity_files=None):
    all_samples = []
    
    # 1. Process Catalogs (96 SBS)
    for f in catalog_files:
        df = pd.read_csv(f)
        channels = format_channel_names(df)
        
        # Transpose so samples are rows
        samples_df = df.drop(columns=['Mutation type', 'Trinucleotide']).T
        samples_df.columns = channels
        samples_df.index.name = 'FullID'
        samples_df = samples_df.reset_index()
        
        # Extract Label and SampleID
        samples_df['Label'] = samples_df['FullID'].apply(lambda x: x.split('::')[0])
        samples_df['SampleID'] = samples_df['FullID'].apply(lambda x: x.split('::')[1] if '::' in x else x)
        
        # Normalize to frequencies
        samples_df[channels] = samples_df[channels].div(samples_df[channels].sum(axis=1) + 1e-9, axis=0)
        all_samples.append(samples_df)
    
    main_df = pd.concat(all_samples, ignore_index=True)
    
    # 2. Process Activities (Optional)
    if activity_files:
        act_list = []
        for f in activity_files:
            adf = pd.read_csv(f)
            if 'Signature' in adf.columns: # Case A: Long
                sigs = adf['Signature']
                adf_t = adf.drop(columns=['Signature']).T
                adf_t.columns = sigs
                adf_t['SampleID'] = adf_t.index.map(lambda x: x.split('::')[1] if '::' in str(x) else x)
                act_list.append(adf_t)
            else: # Case B: Wide
                sig_cols = [c for c in adf.columns if any(p in c for p in ['SBS', 'DBS', 'ID'])]
                id_col = 'Sample Names' if 'Sample Names' in adf.columns else adf.columns[0]
                temp = adf[[id_col] + sig_cols].copy()
                temp.rename(columns={id_col: 'SampleID'}, inplace=True)
                act_list.append(temp)
        
        combined_act = pd.concat(act_list, ignore_index=True).drop_duplicates(subset='SampleID')
        final_df = pd.merge(main_df, combined_act, on='SampleID', how='inner')
    else:
        final_df = main_df

    return final_df

def run_pipeline(df):
    exclude = ['FullID', 'Label', 'SampleID']
    features = [c for c in df.columns if c not in exclude]
    
    X = df[features].values
    le = LabelEncoder()
    y = le.fit_transform(df['Label'])
    
    # Ensure every class has at least 2 samples for stratification
    counts = pd.Series(y).value_counts()
    valid_classes = counts[counts >= 2].index
    mask = np.isin(y, valid_classes)
    X, y = X[mask], y[mask]
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
    
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    models = {
        "LogisticRegression": LogisticRegression(max_iter=1000),
        "RandomForest": RandomForestClassifier(n_estimators=100, n_jobs=-1),
        "HistGB": HistGradientBoostingClassifier(),
        "LinearSVM": LinearSVC(max_iter=2000),
        "MLP": MLPClassifier(hidden_layer_sizes=(100, 50), max_iter=500)
    }
    
    results = []
    for name, clf in models.items():
        print(f"Training {name}...")
        clf.fit(X_train_scaled, y_train)
        preds = clf.predict(X_test_scaled)
        
        # Metrics
        results.append({
            "Model": name, 
            "Accuracy": accuracy_score(y_test, preds), 
            "F1_Macro": f1_score(y_test, preds, average='macro')
        })
        
        # Plotting - Confusion Matrix for top 15 most frequent classes
        top_labels = df['Label'].value_counts().nlargest(15).index
        label_indices = [i for i, label in enumerate(le.classes_) if label in top_labels]
        
        cm = confusion_matrix(y_test, preds, labels=range(len(le.classes_)))
        cm_subset = cm[np.ix_(label_indices, label_indices)]
        
        plt.figure(figsize=(10, 8))
        sns.heatmap(cm_subset, annot=True, fmt='d', xticklabels=top_labels, yticklabels=top_labels, cmap='viridis')
        plt.title(f"Confusion Matrix: {name}")
        plt.tight_layout()
        plt.savefig(f"cm_{name}.png")
        plt.close()

        # FIXED: Pass 'labels' to classification_report to avoid size mismatch
        report = classification_report(
            y_test, 
            preds, 
            labels=range(len(le.classes_)), 
            target_names=le.classes_, 
            output_dict=True
        )
        
        f1_scores = {k: v['f1-score'] for k, v in report.items() if k in le.classes_}
        plt.figure(figsize=(12, 8))
        pd.Series(f1_scores).sort_values().plot(kind='barh')
        plt.title(f"Per-Class F1 Score: {name}")
        plt.tight_layout()
        plt.savefig(f"f1_bars_{name}.png")
        plt.close()

    summary_df = pd.DataFrame(results)
    summary_df.to_csv("model_results_summary.csv", index=False)
    return summary_df

# Usage
PCAWG_wgs_mut = ["./project_data/catalogs/WGS/WGS_PCAWG.96.csv"]
PCAWG_wgs_act = ["./project_data/activities/WGS/WGS_PCAWG.activities.csv"]
data = load_and_preprocess(PCAWG_wgs_mut, PCAWG_wgs_act)
summary = run_pipeline(data)
print(summary)

Training LogisticRegression...


/home/ugteo/.pyenv/versions/3.12.4/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1833: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/ugteo/.pyenv/versions/3.12.4/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1833: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/ugteo/.pyenv/versions/3.12.4/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1833: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.ca

Training RandomForest...


/home/ugteo/.pyenv/versions/3.12.4/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1833: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/ugteo/.pyenv/versions/3.12.4/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1833: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/ugteo/.pyenv/versions/3.12.4/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1833: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.ca

Training HistGB...


...

Training LinearSVM...


/home/ugteo/.pyenv/versions/3.12.4/lib/python3.12/site-packages/sklearn/svm/_base.py:1258: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/ugteo/.pyenv/versions/3.12.4/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1833: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/ugteo/.pyenv/versions/3.12.4/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1833: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/ugteo/.pyenv/versions/3.12.4/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1833: UndefinedMetricWarni

Training MLP...


/home/ugteo/.pyenv/versions/3.12.4/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1833: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/ugteo/.pyenv/versions/3.12.4/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1833: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/ugteo/.pyenv/versions/3.12.4/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1833: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.ca

                Model  Accuracy  F1_Macro
0  LogisticRegression  0.787770  0.599563
1        RandomForest  0.726619  0.493218
2              HistGB  0.784173  0.578367
3           LinearSVM  0.753597  0.538602
4                 MLP  0.773381  0.599461
